In [ ]:
import pandas as pd
import ollama

# Load the dataset
csv_file = "sentiment_test_data.csv"  # Ensure this is the correct path
df = pd.read_csv(csv_file)

# Function to analyze sentiment using Ollama
def analyze_sentiment(text):
    response = ollama.chat(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": f"Analyze the sentiment of this text: '{text}'. Respond with Positive, Negative, or Neutral only."}])
    
    # Extract the content from the response
    sentiment_response = response['message']['content'].strip()
    
    # Remove any unwanted tags or text (e.g., "think:")
    sentiment_response = sentiment_response.replace("think:", "").strip()
    
    # Ensure the response is one of the expected values
    if sentiment_response not in ["Positive", "Negative", "Neutral"]:
        sentiment_response = "Neutral"  # Default to Neutral if the response is unexpected
    
    return sentiment_response

# Apply the model to each row
df["Predicted Sentiment"] = df["Text"].apply(analyze_sentiment)

# Compute accuracy
correct_predictions = (df["Sentiment"] == df["Predicted Sentiment"]).sum()
total_samples = len(df)
accuracy = correct_predictions / total_samples * 100

# Save results to a new CSV
output_file = "sentiment_test_results.csv"
df.to_csv(output_file, index=False)

print(f"Sentiment analysis completed. Results saved to {output_file}")
print(f"Model Accuracy: {accuracy:.2f}%")

In [7]:
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio

# Load the dataset
csv_file = "sentiment_test_data.csv"  # Ensure this is the correct path
df = pd.read_csv(csv_file)

# Function to analyze sentiment asynchronously using Ollama
async def analyze_sentiment_async(text):
    async with aiohttp.ClientSession() as session:
        payload = {
            "model": "deepseek-r1:1.5b",
            "messages": [{"role": "user", "content": f"Analyze the sentiment of this text: '{text}'. Respond with Positive, Negative, or Neutral only."}]
        }
        async with session.post("https://api.ollama.com/chat", json=payload) as response:
            data = await response.json()
            sentiment_response = data['message']['content'].strip()
            sentiment_response = sentiment_response.replace("think:", "").strip()
            if sentiment_response not in ["Positive", "Negative", "Neutral"]:
                sentiment_response = "Neutral"
            return sentiment_response

# Function to analyze sentiment in batches asynchronously
async def analyze_sentiment_batch_async(texts):
    tasks = [analyze_sentiment_async(text) for text in texts]
    return await asyncio.gather(*tasks)

# Required for running asyncio in Jupyter notebooks
nest_asyncio.apply()

# Define batch size
batch_size = 10

# Apply the model to each row in batches
predicted_sentiments = []
for i in range(0, len(df), batch_size):
    batch_texts = df["Text"][i:i+batch_size].tolist()
    batch_sentiments = asyncio.run(analyze_sentiment_batch_async(batch_texts))
    predicted_sentiments.extend(batch_sentiments)

# Add the predicted sentiments to the DataFrame
df["Predicted Sentiment"] = predicted_sentiments

# Compute accuracy
correct_predictions = (df["Sentiment"] == df["Predicted Sentiment"]).sum()
total_samples = len(df)
accuracy = correct_predictions / total_samples * 100

# Save results to a new CSV
output_file = "sentiment_test_results.csv"
df.to_csv(output_file, index=False)

print(f"Sentiment analysis completed. Results saved to {output_file}")
print(f"Model Accuracy: {accuracy:.2f}%")

ClientConnectorDNSError: Cannot connect to host api.ollama.com:443 ssl:default [getaddrinfo failed]

In [5]:
import pandas as pd
from openai import OpenAI

# Load the dataset
csv_file = "sentiment_test_data_200.csv"  # Ensure this is the correct path
df = pd.read_csv(csv_file)

# Create an OpenAI client with your DeepInfra token and endpoint
openai = OpenAI(
    api_key="EbY8YQ5AEq93qODfsgtaf9Vwgzb5KWRW",  # Replace with your actual DeepInfra token
    base_url="https://api.deepinfra.com/v1/openai",
)

# Function to analyze sentiment using OpenAI API
def analyze_sentiment(text):
    response = openai.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",  # Replace with your desired model
        messages=[
            {
                "role": "system",
                "content": """You are a sentiment analysis tool. Your task is to analyze the sentiment of the given text and respond with only one word: Positive, Negative, or Neutral. 
                - "Positive" means the text expresses a favorable, happy, or optimistic tone.
                - "Negative" means the text expresses an unfavorable, sad, or pessimistic tone.
                - "Neutral" means the text is objective, factual, or does not express any strong emotion.
                Examples:
                - Text: "I love this product! It works perfectly." → Sentiment: Positive
                - Text: "This is the worst experience I've ever had." → Sentiment: Negative
                - Text: "The meeting is scheduled for 3 PM." → Sentiment: Neutral"""
            },
            {
                "role": "user",
                "content": f"Analyze the sentiment of the following text and respond with only one word (Positive, Negative, or Neutral): '{text}'"
            },
        ],
        max_tokens=200,  # Limit the response to a single word
        temperature=0.3,  # Lower temperature for more deterministic responses
    )
    
    # Extract the content from the response
    sentiment_response = response.choices[0].message.content.strip()
    index=sentiment_response.find('</think>')+len('</think>')
    sentiment_response=sentiment_response[index:].strip()
    # Ensure the response is one of the expected values
    if sentiment_response not in ["Positive", "Negative", "Neutral"]:
        sentiment_response = "Neutral"  # Default to Neutral if the response is unexpected
    
    return sentiment_response

# Apply the model to each row


In [7]:
df["Predicted Sentiment"] = df["Text"].apply(analyze_sentiment)

# Compute accuracy
correct_predictions = (df["Sentiment"] == df["Predicted Sentiment"]).sum()
total_samples = len(df)
accuracy = correct_predictions / total_samples * 100

# Save results to a new CSV
output_file = "sentiment_test_results.csv"
df.to_csv(output_file, index=False)

print(f"Sentiment analysis completed. Results saved to {output_file}")
print(f"Model Accuracy: {accuracy:.2f}%")

Sentiment analysis completed. Results saved to sentiment_test_results.csv
Model Accuracy: 62.00%


In [6]:
analyze_sentiment('eve is very nice')

'Positive'